In [ ]:
import re
import pandas as pd

In [ ]:
pat = re.compile(r'ROIAlign input shape: \(torch.Size\(\[(\d+), (\d+), (\d+), (\d+)\]\), torch.Size\(\[(\d+), (\d+)\]\), '
                 '(\d+\.\d+), (\d+), (\d+), (\d+), (\w+)')

inputsizes = []
with open('/root/detectron2/roialign.txt', 'r') as file:
    for line in file:
        m = pat.match(line)
        if m:
            inputsizes.append(m.groups())
        else:
            print("ERROR:", line)

inputsizes = pd.DataFrame(inputsizes, columns=['n', 'c', 'h', 'w', 'roi_dimA', 'roi_dimB', 'scale', 'output_h', 'output_w', 'sampling_ratio', 'aligned'])
display(inputsizes)

In [ ]:
unique = inputsizes.groupby(inputsizes.columns.tolist()).size().reset_index(name='numRepeats')#.sort_values(by=['numRepeats'], ascending=False)
display(unique)

In [ ]:
unique.to_csv('/root/detectron2/maskrcnn-roialign-inputsizes.csv', index=False)

In [ ]:
roi_amd = pd.read_csv("/root/detectron2/maskrcnn-roialign-result-rocm-norm.csv")
display(roi_amd)

In [ ]:
roi_nv = pd.read_csv("/root/logs/nvidia/maskrcnn-roialign-result-cuda-norm.csv")
display(roi_nv)

In [ ]:
join_cols = [col for col in roi_amd.columns.tolist() if col not in ['fwd_time(ms)', 'bwd_time(ms)']]

roi_compare = pd.merge(roi_nv, roi_amd, on=join_cols, suffixes=('_cuda', '_rocm'))
display(roi_compare)

In [ ]:
roi_compare['fwd_ratio'] = roi_compare['fwd_time(ms)_rocm'] / roi_compare['fwd_time(ms)_cuda']
roi_compare['bwd_ratio'] = roi_compare['bwd_time(ms)_rocm'] / roi_compare['bwd_time(ms)_cuda']
display(roi_compare)

In [ ]:
roi_compare[((roi_compare['fwd_ratio'] > 2) & (roi_compare['fwd_time(ms)_rocm'] - roi_compare['fwd_time(ms)_cuda'] > 10))]

In [ ]:
roi_compare.to_csv('/root/detectron2/maskrcnn-roialign-compare-norm.csv', index=False)

# NMS

In [ ]:
nms_pat = re.compile(r'\(torch.Size\(\[(\d+), (\d+)\]\), torch.Size\(\[(\d+)\]\), torch.Size\(\[(\d+)\]\), (\d+\.\d+)')

nms_input = []
with open('/root/detectron2/nms.txt', 'r') as file:
    for line in file:
        m = nms_pat.match(line)
        if m:
            nms_input.append(m.groups())
        else:
            print("ERROR:", line)

nms_input = pd.DataFrame(nms_input, columns=['boxes_dimA', 'boxes_dimB', 'scores_dimA', 'lvl_dimA', 'threshold'])
display(nms_input)

In [ ]:
nms_unique = nms_input.groupby(nms_input.columns.tolist()).size().reset_index(name='numRepeats')#.sort_values(by=['numRepeats'], ascending=False)
display(nms_unique)

In [ ]:
nms_unique.to_csv('/root/detectron2/maskrcnn-nms-inputsizes-rocm.csv', index=False)

In [ ]:
nms_amd = pd.read_csv("/root/detectron2/maskrcnn-nms-results-rocm-norm.csv")
display(nms_amd)

In [ ]:
nms_nv = pd.read_csv("/root/logs/nvidia/maskrcnn-nms-results-cuda-norm.csv")
display(nms_nv)

In [ ]:
join_cols = [col for col in nms_amd.columns.tolist() if col not in ['fwd_time(ms)', 'bwd_time(ms)']]

nms_compare = pd.merge(nms_nv, nms_amd, on=join_cols, suffixes=('_cuda', '_rocm'))
display(nms_compare)

In [ ]:
nms_compare['fwd_ratio'] = nms_compare['fwd_time(ms)_rocm'] / nms_compare['fwd_time(ms)_cuda']
display(nms_compare)

In [ ]:
nms_compare[nms_compare['fwd_ratio'] > 2]

In [ ]:
nms_compare.to_csv('/root/detectron2/maskrcnn-nms-compare-norm.csv', index=False)